# Alstom Stock Price Prediction - Model Debugging and Improvement

This notebook provides tools for debugging the prediction model, visualizing its performance, and making improvements.

In [ ]:
import sys
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Add parent directory to path for imports
module_path = str(Path().absolute().parent.parent)
if module_path not in sys.path:
    sys.path.append(module_path)

# Update the import to use the correct module path
from interfaces.cli.stock_predictor_legacy import (
    run, _build_features, _log_forward_return, _build_ensemble_classifier,
    _build_stacked_classifier, _adaptive_threshold, _signals_from_proba
)

import yfinance as yf
from sklearn.metrics import (
    confusion_matrix, classification_report, roc_curve, precision_recall_curve, 
    auc, mean_squared_error, mean_absolute_error
)
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV

# Configure plots
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

## 1. Load and Examine Data

First, let's load the data and examine the dataset structure.

In [ ]:
# Configuration
ticker = "ALO.PA"
years = 5.0
horizon = 5
random_state = 42

# Load data
from datetime import datetime, timedelta
end_date = datetime.today().date()
start_date = end_date - timedelta(days=int(years * 365))

print(f"Downloading data for {ticker} from {start_date} to {end_date}")
data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True, progress=False)
data.tail()

In [ ]:
# Process data and build features
data, features = _build_features(data, start_date, end_date)
print(f"Total features: {len(features)}")
print(f"First 10 features: {features[:10]}")

# Display the first few rows of the processed data
data.tail().T

## 2. Feature Analysis and Importance

Let's analyze the features and their importance for predictions.

In [ ]:
# Target variable
y_ret = _log_forward_return(data['AdjClose'], horizon).dropna()
X = data.loc[y_ret.index, features]

# Create a binary target (direction prediction)
y_direction = (y_ret > 0).astype(int)

print(f"Total samples: {len(X)}")
print(f"Positive class ratio: {y_direction.mean():.2f}")

In [ ]:
# Correlation with target
correlation_with_target = pd.DataFrame({'feature': features, 
                                         'correlation_with_return': [X[f].corr(y_ret) for f in features],
                                         'correlation_with_direction': [X[f].corr(y_direction) for f in features]})

# Sort by absolute correlation with direction
correlation_with_target['abs_corr_direction'] = correlation_with_target['correlation_with_direction'].abs()
correlation_sorted = correlation_with_target.sort_values('abs_corr_direction', ascending=False)

# Plot top 20 correlations
plt.figure(figsize=(12, 10))
top_features = correlation_sorted.head(20)
sns.barplot(x='correlation_with_direction', y='feature', data=top_features)
plt.title('Top 20 Features by Correlation with Price Direction', fontsize=16)
plt.tight_layout()
plt.show()

# Display the correlation table
correlation_sorted.head(20)

In [ ]:
# Feature importance using Random Forest
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=200, random_state=random_state)
rf.fit(X, y_ret)

# Get feature importances
feature_importance = pd.DataFrame({'feature': features, 'importance': rf.feature_importances_})
feature_importance = feature_importance.sort_values('importance', ascending=False)

# Plot top 20 features
plt.figure(figsize=(12, 10))
sns.barplot(x='importance', y='feature', data=feature_importance.head(20))
plt.title('Top 20 Features by Random Forest Importance', fontsize=16)
plt.tight_layout()
plt.show()

# Display the importance table
feature_importance.head(20)

## 3. Model Performance Analysis

Let's evaluate the model performance with detailed metrics and visualizations.

In [ ]:
# Use top features for model training
top_n = 30
top_features = feature_importance['feature'].head(top_n).tolist()
X_reduced = X[top_features]

# Time series cross-validation setup
n_splits = max(3, min(8, len(X) // 80))
tscv = TimeSeriesSplit(n_splits=n_splits, gap=5)

# Collect results
cv_results = {
    'HistGradientBoosting': {'proba': [], 'true': [], 'indices': []},
    'RandomForest': {'proba': [], 'true': [], 'indices': []},
    'Ensemble': {'proba': [], 'true': [], 'indices': []}
}

# Run cross-validation for different models
for model_name in cv_results.keys():
    print(f"\nEvaluating {model_name}...")
    
    all_idx = []
    all_proba = []
    all_true = []
    
    for train_idx, test_idx in tscv.split(X_reduced):
        X_train, X_test = X_reduced.iloc[train_idx], X_reduced.iloc[test_idx]
        y_train, y_test = y_direction.iloc[train_idx], y_direction.iloc[test_idx]
        
        # Select model
        if model_name == 'HistGradientBoosting':
            model = HistGradientBoostingClassifier(max_depth=5, learning_rate=0.05, max_iter=300, random_state=random_state)
        elif model_name == 'RandomForest':
            model = RandomForestClassifier(n_estimators=500, random_state=random_state)
        else:  # Ensemble
            model = _build_ensemble_classifier(random_state=random_state)
            
        # Calibrate
        calibrated = CalibratedClassifierCV(model, cv=3, method='isotonic')
        calibrated.fit(X_train, y_train)
        
        # Predictions
        proba = calibrated.predict_proba(X_test)[:, 1]
        
        all_idx.append(test_idx)
        all_proba.append(proba)
        all_true.append(y_test.values)
    
    # Combine results
    cv_results[model_name]['indices'] = np.concatenate(all_idx)
    cv_results[model_name]['proba'] = np.concatenate(all_proba)
    cv_results[model_name]['true'] = np.concatenate(all_true)
    
    # Performance metrics
    order = np.argsort(cv_results[model_name]['indices'])
    proba_sorted = cv_results[model_name]['proba'][order]
    true_sorted = cv_results[model_name]['true'][order]
    
    # Calculate metrics
    fpr, tpr, _ = roc_curve(true_sorted, proba_sorted)
    roc_auc = auc(fpr, tpr)
    
    # Get predictions with threshold 0.5
    pred = (proba_sorted >= 0.5).astype(int)
    accuracy = np.mean(pred == true_sorted)
    cm = confusion_matrix(true_sorted, pred)
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")
    print("Confusion Matrix:")
    print(cm)

In [ ]:
# Plot ROC curves
plt.figure(figsize=(10, 8))

for model_name, results in cv_results.items():
    order = np.argsort(results['indices'])
    proba_sorted = results['proba'][order]
    true_sorted = results['true'][order]
    
    fpr, tpr, _ = roc_curve(true_sorted, proba_sorted)
    roc_auc = auc(fpr, tpr)
    
    plt.plot(fpr, tpr, lw=2, label=f'{model_name} (AUC = {roc_auc:.3f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('ROC Curves for Different Models', fontsize=16)
plt.legend(loc="lower right", fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
# Plot calibration curves
plt.figure(figsize=(10, 8))

for model_name, results in cv_results.items():
    order = np.argsort(results['indices'])
    proba_sorted = results['proba'][order]
    true_sorted = results['true'][order]
    
    # Create bins for calibration curve
    bins = np.linspace(0, 1, 11)
    bin_centers = bins[:-1] + np.diff(bins) / 2
    bin_indices = np.digitize(proba_sorted, bins) - 1
    bin_indices = np.clip(bin_indices, 0, len(bin_centers) - 1)
    
    bin_sums = np.bincount(bin_indices, weights=true_sorted, minlength=len(bin_centers))
    bin_counts = np.bincount(bin_indices, minlength=len(bin_centers))
    bin_true_probs = np.zeros_like(bin_centers, dtype=float)
    nonzero = bin_counts > 0
    bin_true_probs[nonzero] = bin_sums[nonzero] / bin_counts[nonzero]
    
    plt.plot(bin_centers, bin_true_probs, marker='o', linestyle='-', lw=2, label=model_name)

plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Perfectly calibrated')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('Predicted probability', fontsize=14)
plt.ylabel('True probability in each bin', fontsize=14)
plt.title('Calibration Curves for Different Models', fontsize=16)
plt.legend(loc="lower right", fontsize=12)
plt.grid(True)
plt.show()

## 4. Threshold Optimization

Let's find the optimal threshold for our predictions.

In [ ]:
# Use the ensemble model results for threshold optimization
model_name = 'Ensemble'  # or choose another model
results = cv_results[model_name]
order = np.argsort(results['indices'])
proba_sorted = results['proba'][order]
true_sorted = results['true'][order]

# Test different thresholds
thresholds = np.linspace(0.3, 0.8, 51)
metrics = {'threshold': [], 'accuracy': [], 'precision': [], 'recall': [], 'f1': []}

for threshold in thresholds:
    pred = (proba_sorted >= threshold).astype(int)
    
    # Calculate metrics
    accuracy = np.mean(pred == true_sorted)
    precision = np.sum((pred == 1) & (true_sorted == 1)) / max(1, np.sum(pred == 1))
    recall = np.sum((pred == 1) & (true_sorted == 1)) / max(1, np.sum(true_sorted == 1))
    f1 = 2 * precision * recall / max(1e-10, precision + recall)
    
    metrics['threshold'].append(threshold)
    metrics['accuracy'].append(accuracy)
    metrics['precision'].append(precision)
    metrics['recall'].append(recall)
    metrics['f1'].append(f1)

# Convert to DataFrame
metrics_df = pd.DataFrame(metrics)

# Plot metrics vs threshold
plt.figure(figsize=(12, 8))
for metric in ['accuracy', 'precision', 'recall', 'f1']:
    plt.plot(metrics_df['threshold'], metrics_df[metric], lw=2, label=metric.capitalize())

plt.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5, label='Baseline (0.5)')

# Find best F1 threshold
best_f1_idx = np.argmax(metrics_df['f1'])
best_f1_threshold = metrics_df.loc[best_f1_idx, 'threshold']
plt.axvline(x=best_f1_threshold, color='red', linestyle='--', 
           label=f'Best F1 threshold: {best_f1_threshold:.2f}')

plt.xlabel('Threshold', fontsize=14)
plt.ylabel('Score', fontsize=14)
plt.title('Classification Metrics vs. Threshold', fontsize=16)
plt.legend(loc="best", fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.show()

## 5. Backtest with Optimal Parameters

Now let's run a backtest with our optimized parameters.

In [ ]:
# Get the best threshold from the analysis
best_threshold = metrics_df.loc[best_f1_idx, 'threshold']
print(f"Best threshold from analysis: {best_threshold:.4f}")

# Run with optimized parameters
result = run(
    ticker=ticker,
    years=years,
    horizon=horizon,
    threshold=best_threshold,
    use_ensemble=True,  # Use the ensemble model
    calibrate=True,
    plot=True  # Generate a plot
)

# Display key results
print(f"\nPrediction result for {ticker}:")
print(f"Decision: {result['y_pred']}")
print(f"Probability up: {result['proba_up']:.4f}")
print(f"Adaptive threshold: {result['adaptive_threshold']:.4f}")
print(f"\nLast close: {result['last_close']:.4f}")
print(f"Predicted return: {result['predicted_return']:.4f}")
print(f"Predicted price: {result['predicted_price']:.4f}")

# Display backtest metrics
backtest = result['metrics']['backtest']
print(f"\nBacktest results:")
for key, value in backtest.items():
    print(f"{key}: {value}")

## 6. Error Analysis

Let's analyze where the model is making mistakes.

In [ ]:
# Use the ensemble model results
model_name = 'Ensemble'  # or choose another model
results = cv_results[model_name]
indices = results['indices']
probas = results['proba']
true_values = results['true']

# Create DataFrame with predictions and actual values
error_df = pd.DataFrame({
    'date': X_reduced.index[indices],
    'true_direction': true_values,
    'predicted_proba': probas
})

# Add features to analyze
for feature in top_features[:10]:  # Add top 10 features
    error_df[feature] = X_reduced[feature].values[indices]

# Add prediction with threshold
error_df['predicted_direction'] = (error_df['predicted_proba'] >= best_threshold).astype(int)
error_df['is_correct'] = error_df['predicted_direction'] == error_df['true_direction']
error_df['error_type'] = np.where(error_df['is_correct'], 'Correct', 
                         np.where(error_df['predicted_direction'] == 1, 'False Positive', 'False Negative'))

# Error rate by date
error_df['year_month'] = pd.to_datetime(error_df['date']).dt.to_period('M')
error_by_month = error_df.groupby('year_month').agg(
    accuracy=('is_correct', 'mean'),
    count=('is_correct', 'count')
).reset_index()

# Plot error rate over time
plt.figure(figsize=(14, 6))
plt.bar(error_by_month['year_month'].astype(str), error_by_month['accuracy'], alpha=0.7)
plt.axhline(y=error_df['is_correct'].mean(), color='red', linestyle='--', 
           label=f'Overall accuracy: {error_df["is_correct"].mean():.4f}')
plt.xticks(rotation=90)
plt.xlabel('Month', fontsize=14)
plt.ylabel('Accuracy', fontsize=14)
plt.title('Model Accuracy by Month', fontsize=16)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Feature distributions by error type
feature = top_features[0]  # Change to analyze different features

plt.figure(figsize=(12, 6))
for error_type in ['Correct', 'False Positive', 'False Negative']:
    subset = error_df[error_df['error_type'] == error_type][feature]
    sns.kdeplot(subset, label=f'{error_type} (n={len(subset)})')

plt.xlabel(feature, fontsize=14)
plt.ylabel('Density', fontsize=14)
plt.title(f'Distribution of {feature} by Error Type', fontsize=16)
plt.legend()
plt.grid(True)
plt.show()

## 7. Model Improvement Experiments

Let's try some model improvements and compare them.

In [ ]:
# Function to evaluate a model with specific parameters
def evaluate_model_config(use_ensemble=True, calibrate=True, threshold=0.62, feature_count=30):
    # Subset features if needed
    if feature_count < len(features):
        selected_features = feature_importance['feature'].head(feature_count).tolist()
        X_subset = X[selected_features]
    else:
        X_subset = X
    
    # Time series cross-validation
    n_splits = max(3, min(8, len(X) // 80))
    tscv = TimeSeriesSplit(n_splits=n_splits, gap=5)
    
    all_idx, all_proba, all_true = [], [], []
    
    for train_idx, test_idx in tscv.split(X_subset):
        X_train, X_test = X_subset.iloc[train_idx], X_subset.iloc[test_idx]
        y_train, y_test = y_direction.iloc[train_idx], y_direction.iloc[test_idx]
        
        # Select model type
        if use_ensemble:
            model = _build_ensemble_classifier(random_state=random_state)
        else:
            model = HistGradientBoostingClassifier(max_depth=5, learning_rate=0.05, 
                                                  max_iter=300, random_state=random_state)
        
        # Apply calibration if needed
        if calibrate:
            model = CalibratedClassifierCV(model, cv=3, method='isotonic')
            
        model.fit(X_train, y_train)
        proba = model.predict_proba(X_test)[:, 1]
        
        all_idx.append(test_idx)
        all_proba.append(proba)
        all_true.append(y_test.values)
    
    # Combine results
    idx_oof = np.concatenate(all_idx)
    proba_oof = np.concatenate(all_proba)
    true_oof = np.concatenate(all_true)
    
    # Sort by index
    order = np.argsort(idx_oof)
    proba_sorted = proba_oof[order]
    true_sorted = true_oof[order]
    
    # Calculate metrics
    pred = (proba_sorted >= threshold).astype(int)
    accuracy = np.mean(pred == true_sorted)
    
    precision = np.sum((pred == 1) & (true_sorted == 1)) / max(1, np.sum(pred == 1))
    recall = np.sum((pred == 1) & (true_sorted == 1)) / max(1, np.sum(true_sorted == 1))
    f1 = 2 * precision * recall / max(1e-10, precision + recall)
    
    fpr, tpr, _ = roc_curve(true_sorted, proba_sorted)
    roc_auc = auc(fpr, tpr)
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': roc_auc,
        'config': {
            'use_ensemble': use_ensemble,
            'calibrate': calibrate,
            'threshold': threshold,
            'feature_count': feature_count
        }
    }

# Try different configurations
experiment_configs = [
    {'use_ensemble': True, 'calibrate': True, 'threshold': 0.62, 'feature_count': 30},   # Baseline
    {'use_ensemble': True, 'calibrate': True, 'threshold': best_threshold, 'feature_count': 30},  # Optimized threshold
    {'use_ensemble': True, 'calibrate': True, 'threshold': 0.62, 'feature_count': 50},   # More features
    {'use_ensemble': False, 'calibrate': True, 'threshold': 0.62, 'feature_count': 30},  # No ensemble
    {'use_ensemble': True, 'calibrate': False, 'threshold': 0.62, 'feature_count': 30}   # No calibration
]

# Run experiments
experiment_results = []
for config in experiment_configs:
    print(f"Running experiment with config: {config}")
    result = evaluate_model_config(**config)
    experiment_results.append(result)
    print(f"  Accuracy: {result['accuracy']:.4f}, F1: {result['f1']:.4f}, AUC: {result['roc_auc']:.4f}")

# Convert to DataFrame for comparison
experiment_df = pd.DataFrame(experiment_results)
experiment_df

## 8. Hyperparameter Tuning

Let's tune the hyperparameters for the best model.

In [ ]:
from sklearn.model_selection import ParameterGrid

# Hyperparameters to tune
param_grid = {
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_iter': [200, 300, 500]
}

# Subset features based on best experiment
best_experiment = experiment_df.loc[experiment_df['f1'].idxmax()]
best_feature_count = best_experiment['config']['feature_count']
selected_features = feature_importance['feature'].head(best_feature_count).tolist()
X_subset = X[selected_features]

# Time series CV
n_splits = max(3, min(5, len(X) // 100))  # Fewer splits for speed
tscv = TimeSeriesSplit(n_splits=n_splits, gap=5)

# Results container
tuning_results = []

# Grid search through parameters
for params in ParameterGrid(param_grid):
    f1_scores = []
    
    for train_idx, test_idx in tscv.split(X_subset):
        X_train, X_test = X_subset.iloc[train_idx], X_subset.iloc[test_idx]
        y_train, y_test = y_direction.iloc[train_idx], y_direction.iloc[test_idx]
        
        # Create and train model
        model = HistGradientBoostingClassifier(
            max_depth=params['max_depth'],
            learning_rate=params['learning_rate'],
            max_iter=params['max_iter'],
            random_state=random_state
        )
        
        model.fit(X_train, y_train)
        pred = model.predict(X_test)
        
        # Calculate F1 score
        precision = np.sum((pred == 1) & (y_test.values == 1)) / max(1, np.sum(pred == 1))
        recall = np.sum((pred == 1) & (y_test.values == 1)) / max(1, np.sum(y_test.values == 1))
        f1 = 2 * precision * recall / max(1e-10, precision + recall)
        f1_scores.append(f1)
    
    # Store average F1 score
    tuning_results.append({
        'params': params,
        'mean_f1': np.mean(f1_scores)
    })
    print(f"Params: {params}, Mean F1: {np.mean(f1_scores):.4f}")

# Find best parameters
tuning_df = pd.DataFrame(tuning_results)
best_params = tuning_df.loc[tuning_df['mean_f1'].idxmax()]['params']
print(f"\nBest parameters: {best_params}")
print(f"Best F1 score: {tuning_df['mean_f1'].max():.4f}")

## 9. Final Model with Optimized Parameters

Let's run a prediction with our optimized model configuration.

In [ ]:
# Final optimized parameters
final_params = {
    'ticker': ticker,
    'years': years,
    'horizon': horizon,
    'threshold': best_threshold,  # From threshold optimization
    'use_ensemble': True,  # Based on experiment results
    'calibrate': True,  # Based on experiment results
    'plot': True
}

# Run prediction with optimized parameters
optimized_result = run(**final_params)

# Display key results
print(f"\nOptimized prediction result for {ticker}:")
print(f"Decision: {optimized_result['y_pred']}")
print(f"Probability up: {optimized_result['proba_up']:.4f}")
print(f"Adaptive threshold: {optimized_result['adaptive_threshold']:.4f}")
print(f"\nLast close: {optimized_result['last_close']:.4f}")
print(f"Predicted return: {optimized_result['predicted_return']:.4f}")
print(f"Predicted price: {optimized_result['predicted_price']:.4f}")

# Calculate expected return
expected_return = optimized_result['predicted_price'] / optimized_result['last_close'] - 1
print(f"\nExpected return: {expected_return:.2%}")

## 10. Recommendations for Model Improvement

Based on our analysis, here are some recommendations for improving the model:

1. **Optimal threshold:** Use the optimized threshold value of {best_threshold:.2f} instead of the default 0.62

2. **Ensemble models:** The ensemble approach generally performs better than single models

3. **Calibration:** Always use probability calibration for more reliable decision thresholds

4. **Feature selection:** Focus on the top {best_feature_count} features for better signal-to-noise ratio

5. **Model hyperparameters:** Use the optimized hyperparameters for HistGradientBoostingClassifier

6. **Additional features to consider:**
   - Sector performance indicators (transportation/railway sector ETFs)
   - Broader macroeconomic indicators (interest rates, PMI)
   - News sentiment analysis for Alstom
   - Order book data if available

7. **Volatility-based adjustments:** The adaptive thresholding seems to work well and could be further enhanced

8. **Model ensembling:** Consider adding more diverse models to the ensemble (LightGBM, XGBoost)

9. **Time-based features:** Add more calendar effects and cyclical features